### Seaborn
* Matplotlib기반 고수준 시각화 라이브러리
* 통계 그래프에 특화
* 더 예쁜 기본 스타일

### 주요 플롯 종류
* 관계: scatter, line, regplot
* 분포: hist, kde, violin, box
* 범주: bar, count, point
* 행렬: heatmap, clustermap

### 실무 팁
* 색상 팔레트 활용
* 다차원 데이터 표현(hue, size, style)
* FacetGrid로 조건부 플롯

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# 1. matplotlib 캐시 강제 새로고침
import matplotlib
matplotlib.font_manager._load_fontmanager(try_read_cache=False)

# 2. 모든 폰트 설정을 한글 폰트로 강제 변경
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.sans-serif'] = ['AppleGothic']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.titlesize'] = 'large'
plt.rcParams['axes.labelsize'] = 'medium'
plt.rcParams['xtick.labelsize'] = 'medium'
plt.rcParams['ytick.labelsize'] = 'medium'

# 3. Seaborn 스타일 설정 (폰트 설정 이후에)
sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1.0, rc={"font.family": "AppleGothic"})


In [ ]:
# 1. Seaborn 기본 플롯
print("=== Seaborn 기본 플롯 ===")

# 가상 데이터 생성
np.random.seed(42)
df = pd.DataFrame({
    '그룹': np.random.choice(['A', 'B', 'C'], 300),
    '값': np.random.randn(300) * 10 + 50,
    '카테고리': np.random.choice(['X', 'Y'], 300)
})

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 히스토그램 with KDE
sns.histplot(data=df, x='값', kde=True, bins=30, ax=axes[0, 0], color='skyblue')
axes[0, 0].set_title('히스토그램 + KDE')

# 박스플롯
sns.boxplot(data=df, x='그룹', hue='그룹', y='값', ax=axes[0, 1], palette='Set2')
axes[0, 1].set_title('그룹별 박스플롯')

# 바이올린 플롯
sns.violinplot(data=df, x='그룹', y='값', hue='카테고리',
               split=True, ax=axes[1, 0], palette='muted')
axes[1, 0].set_title('바이올린 플롯')

# 스웜 플롯
sns.swarmplot(data=df, x='그룹', y='값', hue='카테고리',
              ax=axes[1, 1], palette='Set1', size=3)
axes[1, 1].set_title('스웜 플롯')

plt.tight_layout()
plt.show()

In [ ]:
# 2. 상관관계 시각화
print("\n=== 상관관계 시각화 ===")

# 다변수 데이터
np.random.seed(42)
n = 200
data = pd.DataFrame({
    '수학': np.random.randn(n) * 15 + 70,
    '영어': np.random.randn(n) * 12 + 75,
    '과학': np.random.randn(n) * 18 + 68,
    '사회': np.random.randn(n) * 10 + 72
})
# 상관관계 추가
data['영어'] = data['수학'] * 0.7 + np.random.randn(n) * 8
data['과학'] = data['수학'] * 0.5 + np.random.randn(n) * 10

# 페어플롯
g = sns.pairplot(data, diag_kind='kde', plot_kws={'alpha': 0.6, 's': 30},
                 corner=True)
g.fig.suptitle('과목별 성적 상관관계', y=1.02, fontsize=16, fontweight='bold')
plt.show()

# 상관계수 히트맵
plt.figure(figsize=(10, 8))
corr = data.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))  # 상삼각 마스킹
sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('과목 간 상관계수', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# 3. 회귀 플롯
print("\n=== 회귀 플롯 ===")

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# regplot
sns.regplot(data=data, x='수학', y='영어', ax=axes[0],
            scatter_kws={'alpha': 0.5, 's': 50},
line_kws={'color': 'red', 'linewidth': 2})
axes[0].set_title('수학 vs 영어 점수 (회귀선)', fontsize=12, fontweight='bold')
axes[0].grid(alpha=0.3)

# residplot (잔차 플롯)
sns.residplot(data=data, x='수학', y='영어', ax=axes[1],
scatter_kws={'alpha': 0.5, 's': 50})
axes[1].axhline(0, color='red', linestyle='--', linewidth=2)
axes[1].set_title('잔차 플롯', fontsize=12, fontweight='bold')
axes[1].grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# 4. 실전 예제: 고객 세그먼트 분석
print("\n=== 실전: 고객 세그먼트 분석 ===")

# 가상 고객 데이터
np.random.seed(42)
n_customers = 500

df_customers = pd.DataFrame({
    '연령': np.random.normal(35, 12, n_customers).astype(int).clip(18, 70),
    '구매액': np.random.gamma(2, 50, n_customers),
    '방문횟수': np.random.poisson(8, n_customers),
    '회원등급': np.random.choice(['Bronze', 'Silver', 'Gold', 'Platinum'], 
                                n_customers, p=[0.4, 0.3, 0.2, 0.1]),
    '성별': np.random.choice(['남성', '여성'], n_customers),
    '지역': np.random.choice(['서울', '경기', '부산', '기타'], 
                           n_customers, p=[0.3, 0.3, 0.2, 0.2])
})

# 연령과 구매액 상관관계 추가
df_customers['구매액'] = df_customers['구매액'] + df_customers['연령'] * 2

# 데이터 미리보기
print(df_customers.head())
print(f"\n총 고객 수: {len(df_customers)}명")

fig = plt.figure(figsize=(18, 12))

# 1. 회원등급별 구매액 분포
ax1 = plt.subplot(2, 3, 1)
order = ['Bronze', 'Silver', 'Gold', 'Platinum']
sns.violinplot(data=df_customers, x='회원등급', y='구매액', 
               order=order, palette='viridis', ax=ax1)
ax1.set_title('회원등급별 구매액 분포', fontsize=12, fontweight='bold')
ax1.set_ylabel('구매액 (천원)')
ax1.grid(alpha=0.3, axis='y')

# 2. 연령대별 방문횟수
ax2 = plt.subplot(2, 3, 2)
df_customers['연령대'] = pd.cut(df_customers['연령'], 
                               bins=[0, 25, 35, 45, 55, 100], 
                               labels=['20대 이하', '30대', '40대', '50대', '60대 이상'])
sns.barplot(data=df_customers, x='연령대', y='방문횟수', 
            palette='coolwarm', ax=ax2, ci=95, estimator=np.mean)
ax2.set_title('연령대별 평균 방문횟수', fontsize=12, fontweight='bold')
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45)
ax2.set_ylabel('평균 방문횟수')
ax2.grid(alpha=0.3, axis='y')

# 3. 지역별 회원등급 분포
ax3 = plt.subplot(2, 3, 3)
crosstab = pd.crosstab(df_customers['지역'], df_customers['회원등급'])
crosstab[order].plot(kind='bar', stacked=True, ax=ax3, colormap='Set3')
ax3.set_title('지역별 회원등급 분포', fontsize=12, fontweight='bold')
ax3.set_xlabel('지역')
ax3.set_ylabel('고객 수')
ax3.set_xticklabels(ax3.get_xticklabels(), rotation=45)
ax3.legend(title='회원등급', bbox_to_anchor=(1.05, 1), loc='upper left')
ax3.grid(alpha=0.3, axis='y')

# 4. 연령 vs 구매액 산점도 (회원등급별)
ax4 = plt.subplot(2, 3, 4)
colors_map = {'Bronze': 'brown', 'Silver': 'gray', 'Gold': 'gold', 'Platinum': 'purple'}
for grade in order:
    subset = df_customers[df_customers['회원등급'] == grade]
    ax4.scatter(subset['연령'], subset['구매액'], 
                label=grade, alpha=0.6, s=50, color=colors_map[grade])
ax4.set_xlabel('연령')
ax4.set_ylabel('구매액 (천원)')
ax4.set_title('연령-구매액 상관관계', fontsize=12, fontweight='bold')
ax4.legend(title='회원등급')
ax4.grid(alpha=0.3)

# 5. 성별별 구매 패턴
ax5 = plt.subplot(2, 3, 5)
sns.kdeplot(data=df_customers[df_customers['성별']=='남성'], 
            x='구매액', label='남성', fill=True, ax=ax5, color='blue', alpha=0.5)
sns.kdeplot(data=df_customers[df_customers['성별']=='여성'], 
            x='구매액', label='여성', fill=True, ax=ax5, color='red', alpha=0.5)
ax5.set_title('성별 구매액 분포', fontsize=12, fontweight='bold')
ax5.set_xlabel('구매액 (천원)')
ax5.set_ylabel('밀도')
ax5.legend()
ax5.grid(alpha=0.3)

# 6. 방문횟수 vs 구매액
ax6 = plt.subplot(2, 3, 6)
sns.scatterplot(data=df_customers, x='방문횟수', y='구매액', 
                hue='회원등급', size='연령', sizes=(20, 200),
                palette='viridis', ax=ax6, alpha=0.6, hue_order=order)
ax6.set_title('방문횟수-구매액 관계', fontsize=12, fontweight='bold')
ax6.set_xlabel('방문횟수')
ax6.set_ylabel('구매액 (천원)')
ax6.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax6.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# 통계 요약
print("\n=== 회원등급별 통계 ===")
summary = df_customers.groupby('회원등급')[['구매액', '방문횟수', '연령']].agg(['mean', 'std', 'count'])
print(summary.round(2))

print("\n=== 지역별 평균 구매액 ===")
region_summary = df_customers.groupby('지역')['구매액'].agg(['mean', 'std', 'count'])
print(region_summary.round(2))

In [ ]:
# 5. FacetGrid - 조건부 플롯
print("\n=== FacetGrid 예제 ===")

# 시간대별 데이터 추가
df_customers['시간대'] = np.random.choice(['오전', '오후', '저녁'], len(df_customers))

g = sns.FacetGrid(df_customers, col='지역', row='성별', 
                  hue='회원등급', hue_order=order,
                  height=3.5, aspect=1.2,
                  palette='Set2', margin_titles=True)
g.map(plt.scatter, '연령', '구매액', alpha=0.5, s=30)
g.add_legend(title='회원등급')
g.fig.subplots_adjust(top=0.93)
g.fig.suptitle('지역-성별 조합별 고객 분석', fontsize=16, fontweight='bold')
plt.show()

In [ ]:
# 6. 조인트 플롯 (주변 분포 포함)
print("\n=== 조인트 플롯 ===")

# 산점도 + 주변 히스토그램
g = sns.jointplot(data=df_customers, x='연령', y='구매액', 
                  kind='scatter', hue='회원등급', hue_order=order,
                  palette='viridis', height=8, alpha=0.6, 
                  marginal_kws={'bins': 30, 'kde': True})
g.fig.suptitle('연령-구매액 조인트 분포', y=1.02, fontsize=14, fontweight='bold')
plt.show()

# 회귀선 포함
g = sns.jointplot(data=df_customers, x='방문횟수', y='구매액', 
                  kind='reg', height=8, color='teal')
g.fig.suptitle('방문횟수-구매액 회귀분석', y=1.02, fontsize=14, fontweight='bold')
plt.show()

In [ ]:
# 7. 카테고리 플롯
print("\n=== 카테고리 플롯 ===")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Strip Plot
sns.stripplot(data=df_customers, x='회원등급', y='구매액', 
              order=order, hue='성별', palette='Set1', ax=axes[0, 0], 
              dodge=True, alpha=0.5, size=3)
axes[0, 0].set_title('Strip Plot - 개별 데이터 포인트', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('구매액 (천원)')
axes[0, 0].grid(alpha=0.3, axis='y')

# Box + Strip 조합
sns.boxplot(data=df_customers, x='회원등급', y='구매액', 
            order=order, ax=axes[0, 1], palette='pastel')
sns.stripplot(data=df_customers, x='회원등급', y='구매액', 
              order=order, ax=axes[0, 1], color='black', alpha=0.3, size=2)
axes[0, 1].set_title('Box + Strip Plot - 분포와 개별값', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('구매액 (천원)')
axes[0, 1].grid(alpha=0.3, axis='y')

# Point Plot (신뢰구간 포함)
sns.pointplot(data=df_customers, x='연령대', y='구매액', 
              hue='성별', ax=axes[1, 0], palette='Set2', 
              dodge=0.3, markers=['o', 's'], linestyles=['-', '--'],
              capsize=0.1, errwidth=2)
axes[1, 0].set_title('Point Plot - 평균과 95% 신뢰구간', fontsize=12, fontweight='bold')
axes[1, 0].set_xticklabels(axes[1, 0].get_xticklabels(), rotation=45)
axes[1, 0].set_ylabel('평균 구매액 (천원)')
axes[1, 0].grid(alpha=0.3)

# Count Plot
sns.countplot(data=df_customers, x='지역', hue='회원등급', 
              hue_order=order, ax=axes[1, 1], palette='viridis')
axes[1, 1].set_title('지역별 회원등급 고객 수', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('고객 수')
axes[1, 1].grid(alpha=0.3, axis='y')
axes[1, 1].legend(title='회원등급')

plt.tight_layout()
plt.show()

In [ ]:
# 8. 히트맵 - 교차 분석
print("\n=== 히트맵 - 교차 분석 ===")

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 지역 x 회원등급 교차표
ax1 = axes[0]
crosstab1 = pd.crosstab(df_customers['지역'], df_customers['회원등급'], 
                        normalize='index') * 100  # 행 기준 백분율
sns.heatmap(crosstab1[order], annot=True, fmt='.1f', cmap='YlOrRd', 
            ax=ax1, cbar_kws={'label': '비율 (%)'})
ax1.set_title('지역별 회원등급 비율 (%)', fontsize=12, fontweight='bold')
ax1.set_xlabel('회원등급')
ax1.set_ylabel('지역')

# 연령대 x 회원등급 평균 구매액
ax2 = axes[1]
pivot = df_customers.pivot_table(values='구매액', 
                                  index='연령대', 
                                  columns='회원등급', 
                                  aggfunc='mean')
sns.heatmap(pivot[order], annot=True, fmt='.0f', cmap='coolwarm', 
            ax=ax2, cbar_kws={'label': '평균 구매액 (천원)'})
ax2.set_title('연령대-회원등급별 평균 구매액', fontsize=12, fontweight='bold')
ax2.set_xlabel('회원등급')
ax2.set_ylabel('연령대')

plt.tight_layout()
plt.show()

In [ ]:
# 9. 추가 인사이트 - 고가치 고객 식별
print("\n=== 고가치 고객 분석 ===")

# 구매액 상위 20% 고객
high_value_threshold = df_customers['구매액'].quantile(0.8)
df_customers['고객유형'] = df_customers['구매액'].apply(
    lambda x: '고가치' if x >= high_value_threshold else '일반'
)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 고가치 고객의 특성 1: 회원등급 분포
ax1 = axes[0]
high_value = df_customers[df_customers['고객유형'] == '고가치']
grade_dist = high_value['회원등급'].value_counts()[order]
colors = ['brown', 'gray', 'gold', 'purple']
ax1.pie(grade_dist, labels=grade_dist.index, autopct='%1.1f%%', 
        colors=colors, startangle=90)
ax1.set_title('고가치 고객의 회원등급 분포', fontsize=12, fontweight='bold')

# 고가치 고객의 특성 2: 연령 분포 비교
ax2 = axes[1]
sns.kdeplot(data=df_customers[df_customers['고객유형']=='일반'], 
            x='연령', label='일반 고객', fill=True, ax=ax2, color='lightblue', alpha=0.5)
sns.kdeplot(data=df_customers[df_customers['고객유형']=='고가치'], 
            x='연령', label='고가치 고객', fill=True, ax=ax2, color='red', alpha=0.5)
ax2.set_title('연령 분포 비교', fontsize=12, fontweight='bold')
ax2.set_xlabel('연령')
ax2.set_ylabel('밀도')
ax2.legend()
ax2.grid(alpha=0.3)

# 고가치 고객의 특성 3: 방문횟수 비교
ax3 = axes[2]
sns.boxplot(data=df_customers, x='고객유형', y='방문횟수', 
            palette='Set2', ax=ax3)
ax3.set_title('방문횟수 비교', fontsize=12, fontweight='bold')
ax3.set_ylabel('방문횟수')
ax3.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# 통계적 비교
print("\n=== 일반 vs 고가치 고객 비교 ===")
comparison = df_customers.groupby('고객유형')[['연령', '방문횟수', '구매액']].agg(['mean', 'std'])
print(comparison.round(2))

print(f"\n고가치 고객 수: {len(high_value)}명 ({len(high_value)/len(df_customers)*100:.1f}%)")
print(f"고가치 고객 평균 구매액: {high_value['구매액'].mean():.2f}천원")
print(f"일반 고객 평균 구매액: {df_customers[df_customers['고객유형']=='일반']['구매액'].mean():.2f}천원")